In [ ]:
!pip install corner

In [ ]:
import corner
import numpy
import numpy as np
import scipy
import scipy.stats
import pandas
import matplotlib.pyplot as plt

In [ ]:
def SimpleColorPlotFromFunc( 
    Func2D = None,
    xmin = None,
    xmax = None, 
    ymin = None,
    ymax = None, 
    ):

    #Make the list of poitns to plug in from the boundaries:
    x = np.linspace(xmin, xmax, 100)
    y = np.linspace(ymin, ymax, 100)
    X, Y = np.meshgrid(x, y)
    PointsToPlugIn = numpy.vstack([X.ravel(), Y.ravel()])
    PointsToPlugInDataset = PointsToPlugIn.T


    #plug in the list of points:
    FunctionResultValuesForGrid = []
    for Point in PointsToPlugInDataset:
        Value = Func2D(Point)
        FunctionResultValuesForGrid.append(Value)

    #reshape stuff in a confusing way so matplotlib can think of the data like a matrix
    Z = numpy.reshape(FunctionResultValuesForGrid, X.shape).T


    #Actually construct the figure...
    plt.figure()
    heatmap = plt.imshow( 
        numpy.rot90(Z), 
        extent=[xmin, xmax, ymin, ymax] ,
        aspect = 'auto' ,
        interpolation = None,
        )  
    

In [ ]:
def Library_NumpyTwoDimensionalDatasetRandomUniform(
    DomainBoxMinPoint= None,
    DomainBoxMaxPoint= None,
    SampleSize= None,
    PrintExtra = False,
    ):

    Result = None

    ResultTranspose = []
    for ColumnIndex in range(len(DomainBoxMinPoint)):
        Column = numpy.random.uniform( 
            low = DomainBoxMinPoint[ColumnIndex],
            high = DomainBoxMaxPoint[ColumnIndex],
            size = SampleSize
            )
        #print 'Column', Column
        ResultTranspose.append(Column)
    ResultTranspose = numpy.array(ResultTranspose)

    Result = ResultTranspose.T
    return Result

In [ ]:
def Library_CheckIfCoordinateInsideDomainBox(
    Coordinate= None,
    DomainBoxMinimum= None,
    DomainBoxMaximum= None,
    PrintExtra = False,
    ):

    Result = None


    if DomainBoxMinimum is None:
        raise Exception('DomainBoxMinimum is None')
    if DomainBoxMaximum is None:
        raise Exception('DomainBoxMaximum is None')
        

    DomainBoxDimensionCount = len(DomainBoxMinimum)
    assert(DomainBoxDimensionCount == len(DomainBoxMaximum))

    Result = True
    for Dimension in range( DomainBoxDimensionCount ):

        if not ( DomainBoxMinimum[Dimension] <= Coordinate[Dimension] <= DomainBoxMaximum[Dimension] ):
            Result = False
            break

    return Result

#### This is our code from the probability theory activity, which makes a double-peaked joint probablility density function. Then we apply a simulated annealing algorithm (one choice of global optimizer) to find the global peak of this distribution. This is akin to having your MCMC algorithm running with a single walker and 500 steps. Compare the results. 

In [ ]:

Gaussian1Mean = [1,1]
Gaussian2Mean = [2,4]

Gaussian1Covariance = numpy.identity(2)*1
Gaussian2Covariance = numpy.identity(2)*.5


def Gaussian1(ABpoint):
    return scipy.stats.multivariate_normal.pdf( ABpoint, mean = Gaussian1Mean, 
                                               cov = Gaussian1Covariance )

def Gaussian2(ABpoint):
    return scipy.stats.multivariate_normal.pdf( ABpoint, mean = Gaussian2Mean, 
                                               cov = Gaussian2Covariance )

def GaussianMultiModal(ABpoint):
    return Gaussian1(ABpoint) + Gaussian2(ABpoint)

SimpleColorPlotFromFunc( 
    Func2D = GaussianMultiModal,
    xmin = -1,
    xmax = 5, 
    ymin = -1,
    ymax = 5, 
    )
plt.title("Probability Density of A & B", fontsize=30)
plt.ylabel('B',fontsize=40)
plt.xlabel('A',fontsize=40)

In [ ]:
# Simulated Annealing wants to minimize, so we take the negative of our likelihood
def negativeGaussianMultiModal(ABpoint):
    return -1*np.log(GaussianMultiModal(ABpoint))
ScalarFunctionPython= negativeGaussianMultiModal
DomainMinimumPoint= [-5, -5]
DomainMaximumPoint= [5, 5]
# DomainMinimumPoint= [0, 0]
# DomainMaximumPoint= [3, 5]

In [ ]:
DomainMinimumPoint = numpy.array(DomainMinimumPoint)
DomainMaximumPoint = numpy.array(DomainMaximumPoint)



DomainRange = DomainMaximumPoint - DomainMinimumPoint

#Define an affine dependent neighbor range box:
NeighborRange = DomainRange / 2

BoxCenter = DomainMinimumPoint + DomainRange / 2.


TotalTrialCount = 5000 # Change this to increase the max number of trials
SampleCount = 500 # Change this to increase number of accepted samples
Beta = .0005 # Change this to change the "cooling rate"


In [ ]:
def CoolingScheduleFunction( Step ):
    Temp = numpy.exp( - Beta*Step )
    return Temp

In [ ]:
RejectionCount = 0

#Start us off with a single sample from the middle of the box:
StepNumbers             = [0]
Samples                 = [BoxCenter]
FunctionValues          = [ScalarFunctionPython(BoxCenter)]
AcceptanceProbabilities = [1]
Temperatures            = [Beta]



#Now iterate and get smaller values to minimize the function:
CurrentStep = 1
#while CurrentStep < TotalTrialCount:
while (len(Samples) < SampleCount and CurrentStep < TotalTrialCount) :


    #Retain the information on the previously accepted sample:
    PreviousSample          = Samples[-1]
    PreviousFunctionValue   = FunctionValues[-1]


    #First choose a random point in the neighborhood (possible new sample):
    RandomNeighborOnBox = None
    while (
        (RandomNeighborOnBox is None) \
        or Library_CheckIfCoordinateInsideDomainBox( 
            Coordinate = RandomNeighborOnBox,
            DomainBoxMinimum = DomainMinimumPoint,
            DomainBoxMaximum = DomainMaximumPoint,
            ) == False 
        ):
        RandomNeighborOnBox = Library_NumpyTwoDimensionalDatasetRandomUniform(
            DomainBoxMinPoint= PreviousSample - NeighborRange/2,
            DomainBoxMaxPoint= PreviousSample + NeighborRange/2,
            SampleSize= 1,
            )[0]
    #What is the function value at the newly randomly selected point in the box?
    FunctionValue = ScalarFunctionPython(RandomNeighborOnBox)
    #print ('f(',RandomNeighborOnBox,')=', FunctionValue)

    #What temperature are we at now?
    Temp =  CoolingScheduleFunction(CurrentStep )





    #Calculate the probability to accept the new sample:
    ProbabilityToAccept = 0.0
    if FunctionValue < PreviousFunctionValue: #Sample got better
        ProbabilityToAccept = 1.0
    else: #Sample got worse
        ValueDiff = -numpy.abs(FunctionValue - PreviousFunctionValue ) 
        ProbabilityToAccept = numpy.exp( ValueDiff / Temp )

    #Decide if we accept the new sample:
    AcceptSample = False
    ZeroOneUniformValue = numpy.random.uniform(0,1,1)[0]
    if ZeroOneUniformValue < ProbabilityToAccept:
        AcceptSample = True
    else:
        AcceptSample = False


    #If we accept the new sample - keep track of all its information
    if AcceptSample:
        StepNumbers.append(CurrentStep)
        Samples.append(RandomNeighborOnBox)
        AcceptanceProbabilities.append(ProbabilityToAccept)
        Temperatures.append(Temp)
        FunctionValues.append(FunctionValue)
    else:
        RejectionCount += 1


    if CurrentStep %300 == 0: 
        print ('CurrentStep:', CurrentStep , '| CurrentSampleCount:', len(Samples), '|Temp:', Temp, '|Probability', ProbabilityToAccept)
        print ('    ValueDiff', ValueDiff)
        print ('    PreviousSample', PreviousSample)
        print ('    RandomNeighborOnBox', RandomNeighborOnBox)


    CurrentStep+= 1


print ('TotalTrialCount', TotalTrialCount)
print ('AcceptanceCount', len(Samples))
print ('RejectionCount', RejectionCount)

Result = {
    'Samples': numpy.array(Samples),
    'FunctionValues': numpy.array(FunctionValues),
    'AcceptanceProbabilities': numpy.array(AcceptanceProbabilities),
    'Temperatures':numpy.array(Temperatures),
    'StepNumbers':numpy.array(StepNumbers), 
}

In [ ]:
#Lifted right from stack overflow:
def cross_hair(x, y, ax=None, **kwargs):
    if ax is None:
        ax = plt.gca()
    horiz = ax.axhline(y,  **kwargs)
    vert = ax.axvline(x, **kwargs)
    return horiz, vert
SimpleColorPlotFromFunc(  
            Func2D = GaussianMultiModal,      
            xmin =DomainMinimumPoint[0] ,
            xmax=DomainMaximumPoint[0],
            ymin = DomainMinimumPoint[1] ,
            ymax=DomainMaximumPoint[1],
) 
best_guess_peak=np.array(Result['Samples'])[np.array(Result['FunctionValues']).argmin()]
print('Guess:',best_guess_peak,'Truth:',Gaussian2Mean)
fig=plt.gcf()
ax=fig.gca()
ax.scatter(Result['Samples'][:,0],Result['Samples'][:,1])
cross_hair(best_guess_peak[0],best_guess_peak[1],color='red')
plt.xlabel('A',fontsize=20)
plt.ylabel('B',fontsize=20)
plt.draw()

        

In [ ]:
for i in range(len(Result['Temperatures'])):
    plt.scatter(i,Result['Temperatures'][i],color='k')
plt.xlabel('Trial Number',fontsize=20)
plt.ylabel('Temperatures',fontsize=20)

In [ ]:
for i in range(len(Result['AcceptanceProbabilities'])):
    plt.scatter(i,Result['AcceptanceProbabilities'][i],color='k')
plt.xlabel('Trial Number',fontsize=20)
plt.ylabel('Acceptance Probabilities',fontsize=20)

In [ ]:
corner.corner(Result['Samples'],
              truths=Gaussian2Mean,
              labels=['A','B'],
              plot_contours=False,plot_density=False,
              plot_datapoints=True)
plt.show()